<a href="https://colab.research.google.com/github/alberwan/LLM/blob/main/My_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@skt7/build-your-first-ai-assistant-with-open-source-llm-on-google-colab-74bbb6fbca7b

1. Install Libraries

In [1]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

2. LLM setup and configuration:

  1).  Importing libraries: We bring in required modules from the hugging face (hf) transformers library to work with the language model.

  2). Specifing LLM: We specify the hf repository microsoft/Phi-3-mini-4k-instruct to load the instruct LLM from.

  3). Loading model and tokenizer: We load the LLM and its associated tokenizer, which prepares them for use.

  4). Creating pipeline: We set up a hf pipeline object to streamline sending prompts and getting responses from the LLM.



In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Specify the LLM model we'll be using
model_name = "microsoft/Phi-3-mini-4k-instruct"
# Configure for GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create a pipeline object for easy text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


3. Configure LLM generation parameters:

  This code defines settings to manage the LLM’s text generation behavior.

  max_new_tokens: limits the response length.
  return_full_text: ensures we only get the newly generated part of the response.

In [3]:
# Parameters to control LLM's response generation
generation_args = {
    "max_new_tokens": 512,     # Maximum length of the response
    "return_full_text": False,      # Only return the generated text
}
print("Parameters to control LLM's response generation. done!")

Parameters to control LLM's response generation. done!


4. Builing the query function
  This function bundles the process of sending a question (within a conversational context) to the LLM and getting the response back.

  Input: It takes a list of messages that represent the conversation so far.
  Output: It returns the text response generated by the LLM.

In [4]:
def query(messages):
  """Sends a conversation history to the AI assistant and returns the answer.

  Args:
    messages (list): A list of dictionaries, each with "role" and "content" keys.

  Returns:
    str: The answer from the AI assistant.
  """

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

5. Example usage of the above query function

In [5]:
# Example: Math Problem
messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": "What about solving the equation 2x + 3 = 7?"}
]
result = query(messages)
print(result)

 To solve the equation 2x + 3 = 7, follow these steps:

Step 1: Isolate the term with the variable (2x) by subtracting 3 from both sides of the equation.
2x + 3 - 3 = 7 - 3
2x = 4

Step 2: Solve for x by dividing both sides of the equation by 2.
2x / 2 = 4 / 2
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


6. Builing the chat function
  This function creates a chat loop, allowing you to converse with the AI Assistant in real-time.

  Interactive chat: The chat continues between user and assistant, until user types “exit”.
  Conversation history: It maintains a history of the chat, providing the LLM with context for previous messages.

In [6]:
def chat():
  """Enables interactive chat sessions with the AI assistant."""

  # Initialize the conversation with instructions for the AI assistant
  conversation_history = [
      {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."}
  ]

  # Main chat loop
  while True:
    user_input = input("You: ")

    # Check if the user wants to exit the chat
    if user_input.lower() == "exit":
        break

    # Add user's message to the conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Get a response from the AI assistant
    try:
        response = query(conversation_history)
        print("Assistant: ", response)

        # Record the AI assistant's response in the conversation history
        conversation_history.append({"role": "assistant", "content": response})

    except Exception as e:
        print(f"An error occurred: {e}, please try again.")

7. chat

In [7]:
chat()

You: Hi
Assistant:   Hello! How can I assist you today?
You: Where is Emory university located?
Assistant:   Emory University is located in Atlanta, Georgia, United States. Specifically, it is situated in the Druid Hills neighborhood, which is a suburb of Atlanta. The university was founded in 1836 and is known for its strong programs in the liberal arts, sciences, and professional fields. Emory's main campus spans over 1,500 acres and is home to a variety of academic buildings, residential halls, and recreational facilities.
You: Is it a good university?
Assistant:   Evaluating whether Emory University is a "good" university can be subjective and depends on various factors such as academic programs, research opportunities, campus life, and individual goals. However, Emory University is widely recognized for its academic excellence. It is consistently ranked among the top universities in the United States. The university offers a wide range of undergraduate and graduate programs across